# Exploratory Data Analysis using Python
## Movies
_by Virginia Herrero_

## Data collection
The Movie Database (TMDb) is a popular database for movies and TV shows. TMDb provides a web API in the form of so-called RESTful web service. In order to get access to the TMDb API, it is necessary to register in their website and generate an API key. The process is the following:

1. register at https://www.themoviedb.org/account/signup
2. after sucessful registration, go to setting > api
3. generate and memorize the API key

The TMDb API expects a movie ID. All movie IDs can be downloaded from their website as a compressed JSON file. To find more information about it, check the following link: https://developer.themoviedb.org/docs/daily-id-exports. This daily file export is only valid for approximately 3 months, so it is recommended to use the current date to avoid any problems when downloading the files. Note: the date format of the file is MONTH_DAY_YEAR.

To download the daily id export file, the urllib.request library is used. The compressed file is unzipped using gzip. This file is in binary format so it must be converted to text form using UTF-8, and finally save the content on the file in the variable content. 

In [ ]:
# Import all required libraries
import urllib.request
import gzip
import json

In [ ]:
# The API endpoint
url = "http://files.tmdb.org/p/exports/movie_ids_11_14_2024.json.gz"

In [ ]:
# Request to the API
request = urllib.request.Request(url, headers = {"Accept-Encoding": "gzip"})
# Response from the API
response = urllib.request.urlopen(request)
# Decompress the retrieve data
results = gzip.decompress(response.read())
# UTF-8 decoding
content = results.decode("utf-8")

Now, the daily id export file content is saved in the variable **content**. However, the daily id export file is not a valid JSON object, instead each line of this file is. Therefore, each line of the file must be initialized as a JSON object and add it to a list named **data**.

In [ ]:
# Initialize empty data list
data = []

# Iterate through all the lines of the variable content for the length of the file -1 since the last line is empty
for item in content.split("\n")[0:(len(content.split("\n"))-1)]:
    # Parse json object to python object
    parse_json = json.loads(item)
    # Append dictionary to the data list
    data.append(parse_json)

data

Loop through the first 10 rows of the list **data** to see the IDs from the first 10 movies.

In [ ]:
# Loop to obtain the 10 first movies IDs
for element in data[0:10]:
    print(element["id"])

Now the movie IDs and other information are stored in the list named **data**. To access and download all the information about the movies it is necessary to use the TMDb API. For this purpose use the python library TMDbSimple and initialize it with the API key. 

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY = "4e6e31fe089f3cc86ad755423c61d3a7"

Loop through the first 10 movie IDs to take a look at the movie titles.

In [ ]:
# Loop to obtain the 10 first movies titles
for element in data[0:10]:
    print(element["original_title"])

Now all the information of the movies can be accessed with the library tmdbsimple and the TMDb API. The following loop can be used to output the individual keys that can be queried for a movie:

In [ ]:
# Create search class instance
search = tmdb.Search()
# Iterate through the 10 elements of the list data
for element in data[0:10]:
    response = search.movie(query = element["original_title"]) # Search movie by original title
    for result in search.results: # Iterate through the result of the search
        print(result["title"], result["id"], result["release_date"], result["popularity"]) # Print title, id, release data an popularity of each movie

Using the tmdbsimple library, create a new variable called **movies** where all the information of the **data** variable and the cast and crew information of all the movies is stored. 

Loop 1000 times over the following loop to obtained a movie dataset of 1000 items. All the data is stored in a json file named tmdb_movies.json

In [ ]:
# Initialize empty movies list
movies = []

# Loop though the elements of the list data
for element in data[0:1000]:
    # Create a movie class instance
    movie = tmdb.Movies(element["id"])
    response = movie.info() # Get all the primary info available of the movie
    movie_all_info = response.copy() # Create a copy of the dictionary
    response2 = movie.credits() # Get all the info from cast and crew of that movie
    movie_all_info.update(response2) # Add the dictionary containing cast and crew info to the one with the primary information
    movies.append(movie_all_info) # Add the dictionary with all the info to the list movies

movies

Create a file named tmdb_movies.json and store all the information of the variable **movies** in it to use it later as the dataset of this project. 

In [ ]:
movies # Python objects to store in the json file
tmdb_movies = open("tmdb_movies.json", "w")  # Open the json file where all the info from movies will be saved   
json.dump(movies, tmdb_movies)  # Parse python objects into json objects
tmdb_movies.close()  # Close the json file

## Data loading
Load the JSON file **tmdb_movies** as a pandas DataFrame.

Note: the json file to be loaded contains 30498 rows, which is a larger dataset than the one gathered in the data collection section. This larger data set was provided by HLSR Stuttgart. 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Load dataset
df_movies = pd.read_json("tmdb_movies.json")

# Show first 5 lines
df_movies.head(5)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,"{'id': 177062, 'name': 'Blondie Collection', '...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",,3924,tt0029927,[US],en,...,70,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The favorite comic strip of millions at last o...,Blondie,False,7.1,8,"[{'adult': False, 'gender': 1, 'id': 34178, 'k...","[{'adult': False, 'gender': 0, 'id': 34170, 'k..."
1,False,None,None,0,"[{'id': 12, 'name': 'Adventure'}]",,6124,,[DE],de,...,420,[],Released,,"Peter Voss, Thief of Millions",False,0.0,0,"[{'adult': False, 'gender': 2, 'id': 48038, 'k...","[{'adult': False, 'gender': 2, 'id': 2902, 'kn..."
2,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,8773,tt0055747,"[FR, IT, JP, PL]",fr,...,120,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,The Intimate Secrets of Young Lovers,Love at Twenty,False,6.7,48,"[{'adult': False, 'gender': 2, 'id': 1653, 'kn...","[{'adult': False, 'gender': 2, 'id': 2801, 'kn..."
3,False,None,"{'id': 425256, 'name': 'New World Disorder', '...",0,[],http://www.nwdfilms.com,25449,,[US],en,...,69,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,New World Disorder 9: Never Enough,False,6.0,3,"[{'adult': False, 'gender': 2, 'id': 84130, 'k...","[{'adult': False, 'gender': 0, 'id': 112786, '..."
4,False,None,None,0,"[{'id': 10751, 'name': 'Family'}]",,31975,tt1656746,[US],en,...,46,[],Released,,Sesame Street: Elmo Loves You!,True,0.0,0,[],[]


## Data cleaning
Clean and pre-process the data before it undergoes further analysis. Remove unnecessary columns, null values and convert data types.

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30498 entries, 0 to 30497
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  30498 non-null  bool   
 1   backdrop_path          18744 non-null  object 
 2   belongs_to_collection  1972 non-null   object 
 3   budget                 30498 non-null  int64  
 4   genres                 30498 non-null  object 
 5   homepage               3661 non-null   object 
 6   id                     30498 non-null  int64  
 7   imdb_id                30498 non-null  object 
 8   original_language      30498 non-null  object 
 9   original_title         30498 non-null  object 
 10  overview               30498 non-null  object 
 11  popularity             30498 non-null  float64
 12  poster_path            28891 non-null  object 
 13  production_companies   30498 non-null  object 
 14  production_countries   30498 non-null  object 
 15  re

* __Remove columns__

All columns or features that are not necessary for this analysis are dropped from the dataset.

In [ ]:
# Show all column names
df_movies.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [ ]:
# Drop selected columns
df_movies = df_movies.drop(columns = ["adult", "backdrop_path", "belongs_to_collection", "crew", "homepage", "id",
                                      "imdb_id", "original_language", "original_title", "overview", "poster_path", 
                                      "production_companies", "production_countries", "spoken_languages", "status", 
                                      "tagline", "video"])
df_movies.head()

,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,cast
0,10500000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",13.614,1982-06-11,792965326,115.0,E.T. the Extra-Terrestrial,7.4,5597,"[{'cast_id': 20, 'character': 'Elliott', 'cred..."
1,75000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",15.145,1996-06-25,0,145.0,Independence Day,6.8,4880,"[{'cast_id': 18, 'character': 'David Levinson'..."
2,63000000,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",28.268,1999-03-30,463517383,136.0,The Matrix,8.1,12879,"[{'cast_id': 34, 'character': 'Thomas A. Ander..."
3,150000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",17.513,2003-05-15,738599701,138.0,The Matrix Reloaded,6.8,5037,"[{'cast_id': 24, 'character': 'Thomas A. Ander..."
4,150000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",14.032,2003-11-05,424988211,129.0,The Matrix Revolutions,6.5,4537,"[{'cast_id': 1, 'character': 'Thomas A. Anders..."


* __Remove null values__

First, check the distribution of the numerical columns.

75% of the values in the budget and revenue column are null, therefore they will be removed from the datasest.

In [ ]:
# Get descriptive statistics
df_movies.describe()

,budget,popularity,revenue,runtime,vote_average,vote_count
count,3.049800e+04,30498.000000,3.049800e+04,25846.000000,30498.000000,30498.000000
mean,1.471807e+06,1.652612,3.822254e+06,88.170084,4.387835,54.887501
std,1.033231e+07,2.086308,4.000820e+07,40.961693,2.905961,422.898022
min,0.000000e+00,0.600000,0.000000e+00,1.000000,0.000000,0.000000
25%,0.000000e+00,0.600000,0.000000e+00,76.000000,0.000000,0.000000
50%,0.000000e+00,0.948000,0.000000e+00,90.000000,5.300000,2.000000
75%,0.000000e+00,1.529000,0.000000e+00,103.000000,6.500000,8.000000
max,2.800000e+08,49.125000,2.787965e+09,1100.000000,10.000000,16963.000000


In [ ]:
# Columns to replace zero values to nan values
replace_columns = ["budget", "revenue"]
# Replace zero for nan
df_movies[replace_columns] = df_movies[replace_columns].replace(0, np.nan)
# Drop nan values
df_movies = df_movies.dropna(subset = replace_columns)

Print the dataset shape, to check all null values were sucessfully removed. 

In [ ]:
df_movies.shape

(1389, 10)

* __Data types__

Convert object columns into datetime columns to be able to work with years in further analysis. 

In [ ]:
#  Convert string to datetime object
df_movies["release_date"] = pd.to_datetime(df_movies["release_date"])

In [ ]:
# Create a new column that shows the year of the datetime
df_movies["release_year"] = df_movies["release_date"].dt.year
df_movies.head()

,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,cast,release_year
0,10500000.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",13.614,1982-06-11,792965326.0,115.0,E.T. the Extra-Terrestrial,7.4,5597,"[{'cast_id': 20, 'character': 'Elliott', 'cred...",1982.0
2,63000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",28.268,1999-03-30,463517383.0,136.0,The Matrix,8.1,12879,"[{'cast_id': 34, 'character': 'Thomas A. Ander...",1999.0
3,150000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",17.513,2003-05-15,738599701.0,138.0,The Matrix Reloaded,6.8,5037,"[{'cast_id': 24, 'character': 'Thomas A. Ander...",2003.0
4,150000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",14.032,2003-11-05,424988211.0,129.0,The Matrix Revolutions,6.5,4537,"[{'cast_id': 1, 'character': 'Thomas A. Anders...",2003.0
5,31000000.0,"[{'id': 36, 'name': 'History'}, {'id': 10749, ...",5.921,1985-12-20,128499205.0,161.0,Out of Africa,7.2,497,"[{'cast_id': 22, 'character': 'Karen Christenc...",1985.0


In [ ]:
# Drop nan values from the release year column
df_movies = df_movies.dropna(subset = ["release_year"])
df_movies.head()

,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,cast,release_year
0,10500000.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",13.614,1982-06-11,792965326.0,115.0,E.T. the Extra-Terrestrial,7.4,5597,"[{'cast_id': 20, 'character': 'Elliott', 'cred...",1982.0
2,63000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",28.268,1999-03-30,463517383.0,136.0,The Matrix,8.1,12879,"[{'cast_id': 34, 'character': 'Thomas A. Ander...",1999.0
3,150000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",17.513,2003-05-15,738599701.0,138.0,The Matrix Reloaded,6.8,5037,"[{'cast_id': 24, 'character': 'Thomas A. Ander...",2003.0
4,150000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",14.032,2003-11-05,424988211.0,129.0,The Matrix Revolutions,6.5,4537,"[{'cast_id': 1, 'character': 'Thomas A. Anders...",2003.0
5,31000000.0,"[{'id': 36, 'name': 'History'}, {'id': 10749, ...",5.921,1985-12-20,128499205.0,161.0,Out of Africa,7.2,497,"[{'cast_id': 22, 'character': 'Karen Christenc...",1985.0


* __Flatten JSON data structures__

The columns genres and cast contain nested attributes, therefore it is necessary to “flatten” these columns so that the JSON document becomes a list that can be processed as usual. 

In [ ]:
def parse_col_json(column, key):
    """
    Flattens JSON structures within columns
    
    Parameter
    ----------
    column : str
        name of the column to flatten
    
    key : str
        name of the dictionary key to be flattened

    Returns
    -------
    None
    """
    for index, i in zip(df_movies.index, df_movies[column]):
        list1 = []
        for j in range(len(i)):
            list1.append((i[j][key]))
        df_movies.loc[index, column] = json.dumps(list1)

In [ ]:
parse_col_json("genres", "name")
parse_col_json("cast", "name")

**The clean dataset:**

In [ ]:
df_movies.head()

,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,cast,release_year
0,10500000.0,"['Science Fiction', 'Adventure', 'Family', 'Fa...",13.614,1982-06-11,792965326.0,115.0,E.T. the Extra-Terrestrial,7.4,5597,"['Henry Thomas', 'Drew Barrymore', 'Robert Mac...",1982.0
2,63000000.0,"['Action', 'Science Fiction']",28.268,1999-03-30,463517383.0,136.0,The Matrix,8.1,12879,"['Keanu Reeves', 'Laurence Fishburne', 'Carrie...",1999.0
3,150000000.0,"['Adventure', 'Action', 'Thriller', 'Science F...",17.513,2003-05-15,738599701.0,138.0,The Matrix Reloaded,6.8,5037,"['Keanu Reeves', 'Carrie-Anne Moss', 'Laurence...",2003.0
4,150000000.0,"['Adventure', 'Action', 'Thriller', 'Science F...",14.032,2003-11-05,424988211.0,129.0,The Matrix Revolutions,6.5,4537,"['Keanu Reeves', 'Laurence Fishburne', 'Carrie...",2003.0
5,31000000.0,"['History', 'Romance', 'Drama']",5.921,1985-12-20,128499205.0,161.0,Out of Africa,7.2,497,"['Meryl Streep', 'Robert Redford', 'Klaus Mari...",1985.0
